In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from tensorflow.python.client import device_lib
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

%reload_ext nb_black

print("Tensorflow Version:", tf.__version__)
print(device_lib.list_local_devices())
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices("GPU")))

Tensorflow Version: 2.2.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8120631912214316652
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14163087655216954026
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3140655513
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5689932091867122878
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:3b:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2560278782429340021
physical_device_desc: "device: XLA_GPU device"
]
Num GPUs Available: 1


<IPython.core.display.Javascript object>

In [3]:
# Hyperparameters
dropout_rate = 0.2
validation_cut = 10000
vocab_size = 88000
embedding_dim = 16
max_length = 256
trun_type = "post"
oov_tok = "<OOV>"
num_epochs = 50
batch = 512

<IPython.core.display.Javascript object>

In [4]:
(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(
    num_words=vocab_size
)

17465344/17464789 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 0us/step


<IPython.core.display.Javascript object>

In [5]:
word_mapping = keras.datasets.imdb.get_word_index()
word_mapping = {k: v + 3 for k, v in word_mapping.items()}

1646592/1641221 [==============================] - ETA:  - ETA:  - ETA:  - 0s 0us/step


<IPython.core.display.Javascript object>

In [6]:
# Add Invalid keys
word_mapping["<PAD>"] = 0
word_mapping["<START>"] = 1
word_mapping["<UNK>"] = 2
word_mapping["<UNUSED>"] = 3

<IPython.core.display.Javascript object>

In [7]:
reverse_word_index = dict((value, key) for (key, value) in word_mapping.items())


def decode_review(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])


print(test_data[0])
decode_review(test_data[0])

[1, 591, 202, 14, 31, 6, 717, 10, 10, 18142, 10698, 5, 4, 360, 7, 4, 177, 5760, 394, 354, 4, 123, 9, 1035, 1035, 1035, 10, 10, 13, 92, 124, 89, 488, 7944, 100, 28, 1668, 14, 31, 23, 27, 7479, 29, 220, 468, 8, 124, 14, 286, 170, 8, 157, 46, 5, 27, 239, 16, 179, 15387, 38, 32, 25, 7944, 451, 202, 14, 6, 717]


"<START> please give this one a miss br br kristy swanson and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite lacklustre so all you madison fans give this a miss"

<IPython.core.display.Javascript object>

In [8]:
train_data = pad_sequences(
    train_data, maxlen=max_length, truncating=trun_type, value=word_mapping["<PAD>"]
)
test_data = pad_sequences(
    test_data, maxlen=max_length, truncating=trun_type, value=word_mapping["<PAD>"]
)

<IPython.core.display.Javascript object>

In [9]:
x_train = train_data[validation_cut:]
x_val = train_data[:validation_cut]
y_train = train_labels[validation_cut:]
y_val = train_labels[:validation_cut]

<IPython.core.display.Javascript object>

In [84]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dropout(rate=0.6))
model.add(keras.layers.Dense(embedding_dim, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["accuracy"],
)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 16)          1408000   
_________________________________________________________________
global_average_pooling1d_5 ( (None, 16)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 1,408,289
Trainable params: 1,408,289
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [85]:
model.fit(
    x_train,
    y_train,
    batch_size=batch,
    epochs=num_epochs,
    validation_data=(x_val, y_val),
    verbose=2,
)

Epoch 1/50
30/30 - 1s - loss: 0.6925 - accuracy: 0.5511 - val_loss: 0.6914 - val_accuracy: 0.5850
Epoch 2/50
30/30 - 1s - loss: 0.6889 - accuracy: 0.6433 - val_loss: 0.6853 - val_accuracy: 0.7363
Epoch 3/50
30/30 - 1s - loss: 0.6789 - accuracy: 0.7223 - val_loss: 0.6720 - val_accuracy: 0.7213
Epoch 4/50
30/30 - 1s - loss: 0.6594 - accuracy: 0.7522 - val_loss: 0.6490 - val_accuracy: 0.7599
Epoch 5/50
30/30 - 1s - loss: 0.6299 - accuracy: 0.7787 - val_loss: 0.6170 - val_accuracy: 0.7806
Epoch 6/50
30/30 - 1s - loss: 0.5911 - accuracy: 0.8050 - val_loss: 0.5784 - val_accuracy: 0.7996
Epoch 7/50
30/30 - 1s - loss: 0.5453 - accuracy: 0.8203 - val_loss: 0.5372 - val_accuracy: 0.8113
Epoch 8/50
30/30 - 1s - loss: 0.5007 - accuracy: 0.8427 - val_loss: 0.4977 - val_accuracy: 0.8245
Epoch 9/50
30/30 - 1s - loss: 0.4549 - accuracy: 0.8603 - val_loss: 0.4606 - val_accuracy: 0.8353
Epoch 10/50
30/30 - 1s - loss: 0.4152 - accuracy: 0.8766 - val_loss: 0.4290 - val_accuracy: 0.8440
Epoch 11/50
30/30 -

<IPython.core.display.Javascript object>

In [86]:
model.evaluate(np.asarray(test_data), np.asarray(test_labels))

782/782 [==============================] - ETA: 0s - loss: 0.3717 - accuracy: 0.87 - ETA: 1s - loss: 0.3617 - accuracy: 0.86 - ETA: 1s - loss: 0.3857 - accuracy: 0.86 - ETA: 1s - loss: 0.3898 - accuracy: 0.86 - ETA: 1s - loss: 0.3811 - accuracy: 0.86 - ETA: 1s - loss: 0.4019 - accuracy: 0.85 - ETA: 1s - loss: 0.4065 - accuracy: 0.86 - ETA: 1s - loss: 0.4056 - accuracy: 0.86 - ETA: 1s - loss: 0.4062 - accuracy: 0.86 - ETA: 1s - loss: 0.4009 - accuracy: 0.86 - ETA: 1s - loss: 0.4016 - accuracy: 0.86 - ETA: 0s - loss: 0.4077 - accuracy: 0.86 - ETA: 0s - loss: 0.4115 - accuracy: 0.85 - ETA: 0s - loss: 0.4147 - accuracy: 0.85 - ETA: 0s - loss: 0.4144 - accuracy: 0.86 - ETA: 0s - loss: 0.4146 - accuracy: 0.86 - ETA: 0s - loss: 0.4102 - accuracy: 0.86 - ETA: 0s - loss: 0.4096 - accuracy: 0.86 - ETA: 0s - loss: 0.4077 - accuracy: 0.86 - ETA: 0s - loss: 0.4089 - accuracy: 0.86 - ETA: 0s - loss: 0.4086 - accuracy: 0.86 - ETA: 0s - loss: 0.4090 - accuracy: 0.86 - ETA: 0s - loss: 0.4053 - accuracy

[0.406347393989563, 0.8616399765014648]

<IPython.core.display.Javascript object>

In [91]:
class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = keras.Sequential()
        model.add(keras.layers.Embedding(vocab_size, embedding_dim))
        model.add(keras.layers.GlobalAveragePooling1D())
        model.add(
            keras.layers.Dropout(
                rate=hp.Float(
                    "dropout", min_value=0, max_value=1.0, default=0.1, step=0.1
                )
            )
        )
        model.add(
            keras.layers.Dense(
                embedding_dim,
                activation=hp.Choice(
                    "dense_activation",
                    values=["relu", "tanh", "sigmoid"],
                    default="relu",
                ),
            )
        )
        model.add(
            keras.layers.Dropout(
                rate=hp.Float(
                    "dropout", min_value=0, max_value=1.0, default=0.1, step=0.1
                )
            )
        )
        model.add(keras.layers.Dense(1, activation="sigmoid"))

        model.compile(
            loss="binary_crossentropy",
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    "learning_rate",
                    min_value=1e-4,
                    max_value=1e-1,
                    sampling="LOG",
                    default=1e-3,
                )
            ),
            metrics=["accuracy"],
        )
        return model

<IPython.core.display.Javascript object>

In [92]:
input_shape = (x_train.shape[1],)
hypermodel = RegressionHyperModel(input_shape)
tuner_rs = RandomSearch(
    hypermodel,
    objective="val_accuracy",
    seed=42,
    max_trials=10,
    executions_per_trial=2,
    project_name="hyperparameters",
)
tuner_rs.search(x_train, y_train, epochs=10, validation_split=0.2, verbose=2)

Epoch 1/10
375/375 - 9s - loss: 0.6726 - accuracy: 0.5905 - val_loss: 0.5959 - val_accuracy: 0.7890
Epoch 2/10
375/375 - 9s - loss: 0.4801 - accuracy: 0.7970 - val_loss: 0.3785 - val_accuracy: 0.8593
Epoch 3/10
375/375 - 9s - loss: 0.3256 - accuracy: 0.8785 - val_loss: 0.3263 - val_accuracy: 0.8700
Epoch 4/10
375/375 - 9s - loss: 0.2475 - accuracy: 0.9128 - val_loss: 0.3166 - val_accuracy: 0.8740
Epoch 5/10
375/375 - 9s - loss: 0.2070 - accuracy: 0.9343 - val_loss: 0.3257 - val_accuracy: 0.8747
Epoch 6/10
375/375 - 9s - loss: 0.1616 - accuracy: 0.9526 - val_loss: 0.3570 - val_accuracy: 0.8687
Epoch 7/10
375/375 - 10s - loss: 0.1316 - accuracy: 0.9617 - val_loss: 0.3661 - val_accuracy: 0.8770
Epoch 8/10
375/375 - 9s - loss: 0.1089 - accuracy: 0.9698 - val_loss: 0.3902 - val_accuracy: 0.8753
Epoch 9/10
375/375 - 9s - loss: 0.0919 - accuracy: 0.9748 - val_loss: 0.4272 - val_accuracy: 0.8753
Epoch 10/10
375/375 - 9s - loss: 0.0741 - accuracy: 0.9804 - val_loss: 0.4514 - val_accuracy: 0.876

Epoch 1/10
375/375 - 7s - loss: 0.6592 - accuracy: 0.6564 - val_loss: 0.5510 - val_accuracy: 0.8090
Epoch 2/10
375/375 - 8s - loss: 0.4020 - accuracy: 0.8472 - val_loss: 0.3557 - val_accuracy: 0.8593
Epoch 3/10
375/375 - 9s - loss: 0.2527 - accuracy: 0.9078 - val_loss: 0.3189 - val_accuracy: 0.8727
Epoch 4/10
375/375 - 9s - loss: 0.1786 - accuracy: 0.9391 - val_loss: 0.3172 - val_accuracy: 0.8743
Epoch 5/10
375/375 - 9s - loss: 0.1287 - accuracy: 0.9613 - val_loss: 0.3364 - val_accuracy: 0.8737
Epoch 6/10
375/375 - 9s - loss: 0.0929 - accuracy: 0.9741 - val_loss: 0.3455 - val_accuracy: 0.8773
Epoch 7/10
375/375 - 9s - loss: 0.0672 - accuracy: 0.9821 - val_loss: 0.3652 - val_accuracy: 0.8810
Epoch 8/10
375/375 - 9s - loss: 0.0470 - accuracy: 0.9897 - val_loss: 0.3948 - val_accuracy: 0.8783
Epoch 9/10
375/375 - 9s - loss: 0.0355 - accuracy: 0.9912 - val_loss: 0.4266 - val_accuracy: 0.8747
Epoch 10/10
375/375 - 9s - loss: 0.0260 - accuracy: 0.9947 - val_loss: 0.4541 - val_accuracy: 0.8767

Epoch 1/10
375/375 - 8s - loss: 0.5733 - accuracy: 0.6942 - val_loss: 0.3703 - val_accuracy: 0.8537
Epoch 2/10
375/375 - 9s - loss: 0.2954 - accuracy: 0.8887 - val_loss: 0.3174 - val_accuracy: 0.8717
Epoch 3/10
375/375 - 9s - loss: 0.1996 - accuracy: 0.9308 - val_loss: 0.3293 - val_accuracy: 0.8753
Epoch 4/10
375/375 - 10s - loss: 0.1362 - accuracy: 0.9561 - val_loss: 0.3548 - val_accuracy: 0.8777
Epoch 5/10
375/375 - 9s - loss: 0.1019 - accuracy: 0.9684 - val_loss: 0.3898 - val_accuracy: 0.8770
Epoch 6/10
375/375 - 9s - loss: 0.0667 - accuracy: 0.9808 - val_loss: 0.5301 - val_accuracy: 0.8567
Epoch 7/10
375/375 - 9s - loss: 0.0533 - accuracy: 0.9843 - val_loss: 0.5248 - val_accuracy: 0.8693
Epoch 8/10
375/375 - 9s - loss: 0.0436 - accuracy: 0.9867 - val_loss: 0.5529 - val_accuracy: 0.8643
Epoch 9/10
375/375 - 9s - loss: 0.0343 - accuracy: 0.9905 - val_loss: 0.6163 - val_accuracy: 0.8690
Epoch 10/10
375/375 - 9s - loss: 0.0301 - accuracy: 0.9908 - val_loss: 0.6768 - val_accuracy: 0.867

Epoch 1/10
375/375 - 9s - loss: 0.4210 - accuracy: 0.7962 - val_loss: 0.3347 - val_accuracy: 0.8657
Epoch 2/10
375/375 - 9s - loss: 0.1622 - accuracy: 0.9390 - val_loss: 0.4234 - val_accuracy: 0.8373
Epoch 3/10
375/375 - 9s - loss: 0.0677 - accuracy: 0.9768 - val_loss: 0.4890 - val_accuracy: 0.8607
Epoch 4/10
375/375 - 9s - loss: 0.0326 - accuracy: 0.9894 - val_loss: 0.5934 - val_accuracy: 0.8690
Epoch 5/10
375/375 - 9s - loss: 0.0147 - accuracy: 0.9954 - val_loss: 0.7389 - val_accuracy: 0.8630
Epoch 6/10
375/375 - 10s - loss: 0.0148 - accuracy: 0.9948 - val_loss: 0.7973 - val_accuracy: 0.8610
Epoch 7/10
375/375 - 11s - loss: 0.0071 - accuracy: 0.9977 - val_loss: 0.8850 - val_accuracy: 0.8640
Epoch 8/10
375/375 - 9s - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.9911 - val_accuracy: 0.8610
Epoch 9/10
375/375 - 9s - loss: 6.3958e-04 - accuracy: 1.0000 - val_loss: 1.0455 - val_accuracy: 0.8603
Epoch 10/10
375/375 - 9s - loss: 0.0027 - accuracy: 0.9991 - val_loss: 1.3790 - val_accuracy: 

Epoch 1/10
375/375 - 6s - loss: 0.7322 - accuracy: 0.4926 - val_loss: 0.6937 - val_accuracy: 0.4947
Epoch 2/10
375/375 - 6s - loss: 0.7309 - accuracy: 0.4939 - val_loss: 0.6934 - val_accuracy: 0.4947
Epoch 3/10
375/375 - 6s - loss: 0.7351 - accuracy: 0.4997 - val_loss: 0.6948 - val_accuracy: 0.5053
Epoch 4/10
375/375 - 6s - loss: 0.7117 - accuracy: 0.4951 - val_loss: 0.6933 - val_accuracy: 0.4947
Epoch 5/10
375/375 - 7s - loss: 0.7041 - accuracy: 0.4951 - val_loss: 0.6931 - val_accuracy: 0.4947
Epoch 6/10
375/375 - 9s - loss: 0.6987 - accuracy: 0.5024 - val_loss: 0.6942 - val_accuracy: 0.5053
Epoch 7/10
375/375 - 9s - loss: 0.6948 - accuracy: 0.5048 - val_loss: 0.6932 - val_accuracy: 0.5053
Epoch 8/10
375/375 - 10s - loss: 0.6947 - accuracy: 0.5039 - val_loss: 0.6962 - val_accuracy: 0.5053
Epoch 9/10
375/375 - 9s - loss: 0.6955 - accuracy: 0.5027 - val_loss: 0.6933 - val_accuracy: 0.5053
Epoch 10/10
375/375 - 9s - loss: 0.6951 - accuracy: 0.4886 - val_loss: 0.6934 - val_accuracy: 0.505

...

Epoch 1/10
375/375 - 9s - loss: 0.4004 - accuracy: 0.8122 - val_loss: 0.3462 - val_accuracy: 0.8523
Epoch 2/10
375/375 - 9s - loss: 0.1436 - accuracy: 0.9473 - val_loss: 0.4289 - val_accuracy: 0.8430
Epoch 3/10
375/375 - 6s - loss: 0.0578 - accuracy: 0.9795 - val_loss: 0.5335 - val_accuracy: 0.8593
Epoch 4/10
375/375 - 6s - loss: 0.0293 - accuracy: 0.9902 - val_loss: 0.9021 - val_accuracy: 0.8227
Epoch 5/10
375/375 - 6s - loss: 0.0305 - accuracy: 0.9879 - val_loss: 0.7159 - val_accuracy: 0.8623
Epoch 6/10
375/375 - 6s - loss: 0.0214 - accuracy: 0.9921 - val_loss: 0.8210 - val_accuracy: 0.8650
Epoch 7/10
375/375 - 6s - loss: 0.0203 - accuracy: 0.9920 - val_loss: 0.8768 - val_accuracy: 0.8603
Epoch 8/10
375/375 - 7s - loss: 0.0103 - accuracy: 0.9963 - val_loss: 0.9784 - val_accuracy: 0.8597
Epoch 9/10
375/375 - 7s - loss: 0.0146 - accuracy: 0.9956 - val_loss: 0.9717 - val_accuracy: 0.8557
Epoch 10/10
375/375 - 7s - loss: 0.0106 - accuracy: 0.9962 - val_loss: 0.8554 - val_accuracy: 0.8457

Epoch 1/10
375/375 - 9s - loss: 0.6924 - accuracy: 0.5114 - val_loss: 0.6888 - val_accuracy: 0.5143
Epoch 2/10
375/375 - 9s - loss: 0.6740 - accuracy: 0.5980 - val_loss: 0.6370 - val_accuracy: 0.7897
Epoch 3/10
375/375 - 9s - loss: 0.5731 - accuracy: 0.7342 - val_loss: 0.4839 - val_accuracy: 0.8417
Epoch 4/10
375/375 - 9s - loss: 0.4543 - accuracy: 0.8111 - val_loss: 0.3936 - val_accuracy: 0.8560
Epoch 5/10
375/375 - 9s - loss: 0.3785 - accuracy: 0.8513 - val_loss: 0.3517 - val_accuracy: 0.8610
Epoch 6/10
375/375 - 9s - loss: 0.3305 - accuracy: 0.8729 - val_loss: 0.3329 - val_accuracy: 0.8667
Epoch 7/10
375/375 - 10s - loss: 0.2914 - accuracy: 0.8910 - val_loss: 0.3222 - val_accuracy: 0.8717
Epoch 8/10
375/375 - 9s - loss: 0.2697 - accuracy: 0.8982 - val_loss: 0.3190 - val_accuracy: 0.8727
Epoch 9/10
375/375 - 9s - loss: 0.2436 - accuracy: 0.9097 - val_loss: 0.3194 - val_accuracy: 0.8733
Epoch 10/10
375/375 - 9s - loss: 0.2201 - accuracy: 0.9192 - val_loss: 0.3233 - val_accuracy: 0.873

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


<IPython.core.display.Javascript object>

In [93]:
# Show a summary of the search
tuner_rs.results_summary()
print(tuner_rs.get_best_hyperparameters())

# Retrieve the best model.
best_model = tuner_rs.get_best_models(num_models=1)[0]

# Save the best model.
best_model.save("./Models/tuned_imdb.h5")
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          1408000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,408,289
Trainable params: 1,408,289
Non-trainable params: 0
______________________________________________

<IPython.core.display.Javascript object>

In [94]:
loss, accuracy = best_model.evaluate(np.asarray(test_data), np.asarray(test_labels))

782/782 [==============================] - ETA: 0s - loss: 0.3443 - accuracy: 0.90 - ETA: 1s - loss: 0.3517 - accuracy: 0.86 - ETA: 1s - loss: 0.3546 - accuracy: 0.85 - ETA: 1s - loss: 0.3682 - accuracy: 0.86 - ETA: 1s - loss: 0.3676 - accuracy: 0.86 - ETA: 1s - loss: 0.3606 - accuracy: 0.86 - ETA: 1s - loss: 0.3734 - accuracy: 0.86 - ETA: 1s - loss: 0.3745 - accuracy: 0.86 - ETA: 1s - loss: 0.3798 - accuracy: 0.86 - ETA: 1s - loss: 0.3814 - accuracy: 0.86 - ETA: 1s - loss: 0.3849 - accuracy: 0.86 - ETA: 1s - loss: 0.3881 - accuracy: 0.86 - ETA: 1s - loss: 0.3817 - accuracy: 0.86 - ETA: 1s - loss: 0.3832 - accuracy: 0.86 - ETA: 1s - loss: 0.3852 - accuracy: 0.86 - ETA: 1s - loss: 0.3892 - accuracy: 0.86 - ETA: 1s - loss: 0.3957 - accuracy: 0.86 - ETA: 1s - loss: 0.3955 - accuracy: 0.86 - ETA: 1s - loss: 0.3960 - accuracy: 0.86 - ETA: 0s - loss: 0.3955 - accuracy: 0.86 - ETA: 0s - loss: 0.3964 - accuracy: 0.86 - ETA: 0s - loss: 0.3947 - accuracy: 0.86 - ETA: 0s - loss: 0.3947 - accuracy

<IPython.core.display.Javascript object>

In [95]:
print("Model Accuracy:", accuracy * 100, "%")

Model Accuracy: 86.15599870681763 %


<IPython.core.display.Javascript object>

In [96]:
def encode(s):
    encoded = [1]
    sentence = s.split()
    for word in sentence:
        if word.lower() in word_mapping:
            encoded.append(word_mapping[word.lower()])
        else:
            encoded.append(2)
    return encoded

<IPython.core.display.Javascript object>

In [101]:
sentence_test = "This was bad"
real_encode = encode(sentence_test)
real_padded = pad_sequences(
    [real_encode], maxlen=max_length, truncating=trun_type, value=word_mapping["<PAD>"]
)
print("Review: ", sentence_test)
predict = best_model.predict([real_padded])
print("Prediction: ", str(predict[0][0]))

Review:  This was bad
Prediction:  0.45199072


<IPython.core.display.Javascript object>